#  This is a shared notebook for Project I - Group 3

In [ ]:
#Dependencies
import pandas as pd

In [ ]:
##Amanda's code starts here ##

In [ ]:
#List of the columns we want to read
col_list = ["month_of_death","day_of_week_of_death","current_data_year",
                              "manner_of_death","358_cause_recode","sex","detail_age"]

#Read all of the CSVs into dataframes
data_2005 = pd.read_csv('Resources/2005_data.csv', usecols=col_list)  
data_2006 = pd.read_csv('Resources/2006_data.csv', usecols=col_list)  
data_2007 = pd.read_csv('Resources/2007_data.csv', usecols=col_list)  
data_2008 = pd.read_csv('Resources/2008_data.csv', usecols=col_list)  
data_2009 = pd.read_csv('Resources/2009_data.csv', usecols=col_list)  
data_2010 = pd.read_csv('Resources/2010_data.csv', usecols=col_list)  
data_2011 = pd.read_csv('Resources/2011_data.csv', usecols=col_list)  
data_2012 = pd.read_csv('Resources/2012_data.csv', usecols=col_list)  
data_2013 = pd.read_csv('Resources/2013_data.csv', usecols=col_list)  
data_2014 = pd.read_csv('Resources/2014_data.csv', usecols=col_list)  
data_2015 = pd.read_csv('Resources/2015_data.csv', usecols=col_list)  

In [ ]:
combined_df = data_2005.append([data_2006, data_2007, data_2008, data_2009, data_2010, data_2011,\
                                data_2012, data_2013, data_2014, data_2015])
combined_df

In [ ]:
#Filter to only deaths related to cars
car_death_data = (combined_df["358_cause_recode"] >=385) & (combined_df["358_cause_recode"] <=398)

car_death_data = combined_df[car_death_data]

car_death_data.head()

In [ ]:
#Clean up the dataframe so that it is readable
month_dict = {
      1:"January",
      2:"February",
      3:"March",
      4:"April",
      5:"May",
      6:"June",
      7:"July",
      8:"August",
      9:"September",
      10:"October",
      11:"November",
      12:"December"}
      
day_of_week_dict = {
      1:"Sunday",
      2:"Monday",
      3:"Tuesday",
      4:"Wednesday",
      5:"Thursday",
      6:"Friday",
      7:"Saturday",
      9:"Unknown"}

manner_of_death_dict = {
      1:"Accident",
      2:"Suicide",
      3:"Homicide",
      4:"Pending investigation",
      5:"Could not determine",
      6:"Self-Inflicted",
      7:"Natural"}
#       "Blank":"Not specified"}

cause_recode_dict = {
      385:" 385- Motor vehicle accidents",
      386:" 386- Pedestrian involved in collision with motor vehicle",
      387:" 387- Pedalcyclist involved in collision with motor vehicle",
      388:" 388- Motorcyclist involved in any accident except collision with railway train",
      389:" 389- Motor vehicle accident involving collision with railway train",
      390:" 390- Motorcyclist involved in collision with railway train",
      391:" 391- Other motor vehicle accident involving collision with railway train",
      392:" 392- Occupant of motor vehicle involved in collision with other (non- motorized) road vehicle, streetcar, animal or pedestrian",
      393:" 393- Occupant of car, pickup truck or van involved in collision with other motor vehicle",
      394:" 394- Occupant of heavy transport vehicle or bus involved in collision with other motor vehicle",
      395:" 395- Occupant of motor vehicle involved in non-collision accident",
      396:" 396- Occupant of special-use motor vehicle involved in any accident",
      397:" 397- Other and unspecified motor vehicle accidents",
      398:" 398- Streetcar accidents"}
    

clean_df = car_death_data.replace({"month_of_death": month_dict, 
                        "day_of_week_of_death": day_of_week_dict,
                        "manner_of_death": manner_of_death_dict,
                        "358_cause_recode": cause_recode_dict})

clean_df

In [ ]:
##Amanda's code ends here##